In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import BalancedSampler

In [4]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}

args = Args()
device = args.device

In [5]:
res, drug_feature, exprs, mut, cna, null_mask = load_data(args)

load gdsc2
response matrix (res) shape: (153, 654)
exprs shape: (1084, 19562)
mut shape: (1084, 18099)
cna shape: (1084, 24502)
654
exprs shape: (654, 19562)
mut shape: (654, 18099)
cna shape: (654, 24502)
drug_feature shape: (153, 920)
response matrix (res) shape: (654, 153)
null_mask shape: (654, 153)


In [6]:
res

,"1,3-Bis(2-chloroethyl)-1-nitrosourea",2-fluoroAraA (fludarabine),5-Fluorouracil,681640,AGI-5198,AMG-319,AT13148,AZ6102,AZ960,AZD1208,...,Venetoclax,Vincristine,Vorinostat,WEHI-539,WIKI4,WZ4003,Wnt-C59,XAV-939,YK 4-279,ZM-447439
22RV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23132-87,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42-MG-BA,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
639-V,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSU-DLCL2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WSU-NHL,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
YH-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YT,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
adj_coo = sp.coo_matrix(res)
pos_edges = np.vstack((adj_coo.row, adj_coo.col)).T

# ネガティブエッジ候補作成
adj_dense = adj_coo.toarray()

# ポジティブ（接続ありかつ null_mask==0）
pos_mask = np.logical_and(adj_dense == 1, null_mask == 0)
pos_row, pos_col = np.where(pos_mask)
pos_edges = np.vstack((pos_row, pos_col)).T
pos_labels = np.ones(len(pos_edges), dtype=int)

# ネガティブ（接続なしかつ null_mask==0）
neg_mask = np.logical_and(adj_dense == 0, null_mask == 0)
neg_row, neg_col = np.where(neg_mask)
neg_edges = np.vstack((neg_row, neg_col)).T
neg_labels = np.zeros(len(neg_edges), dtype=int)

# 統合
all_edges = np.vstack((pos_edges, neg_edges))
all_labels = np.concatenate((pos_labels, neg_labels))

In [8]:
exprs.shape

(654, 19562)

In [9]:
len(all_labels)

10113

In [10]:
k = 5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()

for i, (train_idx, test_idx) in enumerate(kf.split(all_edges)):
    train_edges = all_edges[train_idx]
    train_labels = all_labels[train_idx]
    test_edges = all_edges[test_idx]
    test_labels = all_labels[test_idx]

    sampler = BalancedSampler(
        edge_train=train_edges,
        label_train=train_labels,
        edge_test=test_edges,
        label_test=test_labels,
        adj_shape=res.shape,
    )
    
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.769224 auc:0.4803
epoch:  20 loss:0.639434 auc:0.7795
epoch:  40 loss:0.581753 auc:0.8132
epoch:  60 loss:0.478333 auc:0.8528
epoch:  80 loss:0.421202 auc:0.8668
epoch: 100 loss:0.398488 auc:0.8752
epoch: 120 loss:0.381276 auc:0.8823
epoch: 140 loss:0.358985 auc:0.8870
epoch: 160 loss:0.334100 auc:0.8885
epoch: 180 loss:0.305022 auc:0.8916
epoch: 200 loss:0.259966 auc:0.8956
epoch: 220 loss:0.223580 auc:0.9020
epoch: 240 loss:0.193644 auc:0.9077
epoch: 260 loss:0.174075 auc:0.9105
epoch: 280 loss:0.156769 auc:0.9147
epoch: 300 loss:0.151464 auc:0.9139
epoch: 320 loss:0.142525 auc:0.9150
epoch: 340 loss:0.141492 auc:0.9163
epoch: 360 loss:0.133772 auc:0.9154
epoch: 380 loss:0.128334 auc:0.9162
epoch: 400 loss:0.127711 auc:0.9151
epoch: 420 loss:0.123536 auc:0.9160
epoch: 440 loss:0.119003 auc:0.9170
epoch: 460 loss:0.118306 auc:0.9173
epoch: 480 loss:0.115565 auc:0.9174
epoch: 500 loss:0.115539 auc:0.9161
Fit finished.
epoch:   0 loss:0.729527 auc:0.4614
epoch:  20 los

In [11]:
true_datas.to_csv(f"true_{args.data}.csv")
predict_datas.to_csv(f"pred_{args.data}.csv")